In [18]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys
sys.path.append('../')
import utils
import wiki_utils
%matplotlib inline

In [19]:
from torchtext.datasets import language_modeling, WikiText2
from torchtext.data import BPTTIterator, Field

In [20]:
tokenizer = lambda x: list(x)

In [21]:
TEXT = Field(sequential=True, 
             tokenize=tokenizer, 
             include_lengths=False, 
             use_vocab=True)

In [22]:
train, val, test = WikiText2.splits(text_field=TEXT,
                                    path='./wikitext/', train='train.txt',
                                    validation='valid.txt', test='test.txt')

In [23]:
sequence_length = 30
batch_size = eval_batch_size = 128

grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

In [24]:
TEXT.build_vocab(train)

In [25]:
train_iter, val_iter, test_iter = BPTTIterator.splits(
    (train, val, test),
    batch_size=batch_size,
    bptt_len=sequence_length,
    repeat=False, device='cuda')

In [26]:
b = next(iter(train_iter)); vars(b).keys()

dict_keys(['batch_size', 'dataset', 'fields', 'text', 'target'])

In [27]:
b.text[:5, :3]

tensor([[ 2,  9, 16],
        [30, 21, 21],
        [ 2,  2,  4],
        [34, 20, 11],
        [ 2,  5, 10]], device='cuda:0')

In [28]:
b.target[:5, :3]

tensor([[30, 21, 21],
        [ 2,  2,  4],
        [34, 20, 11],
        [ 2,  5, 10],
        [72, 10,  2]], device='cuda:0')

In [29]:
b = next(iter(val_iter)); vars(b).keys()

dict_keys(['batch_size', 'dataset', 'fields', 'text', 'target'])

In [30]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [31]:
ntokens = len(TEXT.vocab.itos)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3)
criterion = nn.CrossEntropyLoss()

In [32]:
def evaluate(data_loader):
    model.cuda()
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab.itos)
    hidden = model.init_hidden(eval_batch_size)
    for i, data in enumerate(data_loader):
        output, hidden = model(data.text.cuda())
        output_flat = output.view(-1, ntokens)
        #print(output_flat.shape)
        #print(data.target.view(-1).shape)
        total_loss += len(data) * criterion(output_flat, data.target.view(-1)).item()
    return total_loss / len(data_loader)

In [33]:
def train():
    model.cuda()
    model.train()
    total_loss = 0
    ntokens = len(TEXT.vocab.itos)
    for batch, data in enumerate(train_iter):
        model.zero_grad()
        output, hidden = model(data.text.cuda())
        #print(output.shape)
        #print(data.target.shape)
        loss = criterion(output.view(-1, ntokens), data.target.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_iter) // sequence_length, lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [34]:
def generate(n=50, temp=1.):
    model.cuda()
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().cuda()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ''.join(out)

In [35]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 10):
    train()
    val_loss = evaluate(val_iter)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 ″!’đRëμpw्;礮&αṅÆოHOñe!đ×ç½-äṭNÎ—γhµĐ⁄Áóณ³µცà+გÞī大ჯ 

| epoch   1 |   100/   93 batches | lr 4.00 | loss  3.60 | ppl    36.59
| epoch   1 |   200/   93 batches | lr 4.00 | loss  3.28 | ppl    26.64
| epoch   1 |   300/   93 batches | lr 4.00 | loss  3.25 | ppl    25.78
| epoch   1 |   400/   93 batches | lr 4.00 | loss  3.22 | ppl    25.11
| epoch   1 |   500/   93 batches | lr 4.00 | loss  3.20 | ppl    24.45
| epoch   1 |   600/   93 batches | lr 4.00 | loss  3.06 | ppl    21.39
| epoch   1 |   700/   93 batches | lr 4.00 | loss  2.97 | ppl    19.45
| epoch   1 |   800/   93 batches | lr 4.00 | loss  2.88 | ppl    17.87
| epoch   1 |   900/   93 batches | lr 4.00 | loss  2.81 | ppl    16.58
| epoch   1 |  1000/   93 batches | lr 4.00 | loss  2.73 | ppl    15.38
| epoch   1 |  1100/   93 batches | lr 4.00 | loss  2.64 | ppl    14.00
| epoch   1 |  1200/   93 batches | lr 4.00 | loss  2.58 | ppl    13.14
| epoch   1 |  1300/   93 batches | lr 4.00 | loss  2.52 | ppl    12.45
| 

| epoch   4 |  1400/   93 batches | lr 4.00 | loss  1.74 | ppl     5.71
| epoch   4 |  1500/   93 batches | lr 4.00 | loss  1.75 | ppl     5.75
| epoch   4 |  1600/   93 batches | lr 4.00 | loss  1.75 | ppl     5.76
| epoch   4 |  1700/   93 batches | lr 4.00 | loss  1.75 | ppl     5.73
| epoch   4 |  1800/   93 batches | lr 4.00 | loss  1.75 | ppl     5.73
| epoch   4 |  1900/   93 batches | lr 4.00 | loss  1.75 | ppl     5.78
| epoch   4 |  2000/   93 batches | lr 4.00 | loss  1.74 | ppl     5.72
| epoch   4 |  2100/   93 batches | lr 4.00 | loss  1.75 | ppl     5.74
| epoch   4 |  2200/   93 batches | lr 4.00 | loss  1.75 | ppl     5.74
| epoch   4 |  2300/   93 batches | lr 4.00 | loss  1.75 | ppl     5.75
| epoch   4 |  2400/   93 batches | lr 4.00 | loss  1.74 | ppl     5.68
| epoch   4 |  2500/   93 batches | lr 4.00 | loss  1.74 | ppl     5.68
| epoch   4 |  2600/   93 batches | lr 4.00 | loss  1.75 | ppl     5.73
| epoch   4 |  2700/   93 batches | lr 4.00 | loss  1.74 | ppl  

| epoch   7 |  2800/   93 batches | lr 4.00 | loss  1.66 | ppl     5.25
-----------------------------------------------------------------------------------------
| end of epoch   7 | valid loss 185.76 | valid ppl 470622635350716199974619780004116031634583466044229020515765394505027156229226496.00
-----------------------------------------------------------------------------------------
sample:
  and can I : C. Jury N @,@ 600 <unk> ( 8 – 14 – 0  

| epoch   8 |   100/   93 batches | lr 4.00 | loss  1.68 | ppl     5.37
| epoch   8 |   200/   93 batches | lr 4.00 | loss  1.66 | ppl     5.26
| epoch   8 |   300/   93 batches | lr 4.00 | loss  1.66 | ppl     5.28
| epoch   8 |   400/   93 batches | lr 4.00 | loss  1.67 | ppl     5.30
| epoch   8 |   500/   93 batches | lr 4.00 | loss  1.67 | ppl     5.29
| epoch   8 |   600/   93 batches | lr 4.00 | loss  1.66 | ppl     5.27
| epoch   8 |   700/   93 batches | lr 4.00 | loss  1.66 | ppl     5.28
| epoch   8 |   800/   93 batches | lr 4.00 | 

In [36]:
t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w') as outf:
    outf.write(t075)
with open('./generated1.txt', 'w') as outf:
    outf.write(t1)
with open('./generated15.txt', 'w') as outf:
    outf.write(t15)